# Running a 5-Point Calibration Scan Without OSO Using BITE

This notebook adapts the [Five Point Calibration Scan Controls](https://gitlab.com/ska-telescope/ska-jupyter-scripting/-/blob/main/notebooks/observing/MID_five_point_calibration_scan_controls.ipynb?ref_type=heads) notebook to work with PSI. Running this notebook requires a PSI namespace to be spun up with the CI argument `TMC_ENABLED` set to `true`.

## 1 Setup

## 1.1 Environment Setup

Start by importing all the libraries we'll be using for this notebook.

In [1]:
import json
import os
import random
import sys
import time
from datetime import date
from time import sleep

from tango import DevFailed, DeviceProxy

Until notebook clean up is merged need to use this for waiting for state

In [2]:
spinner = ["⣾", "⣽", "⣻", "⢿", "⡿", "⣟", "⣯", "⣷"]


def wait_for_state(device: DeviceProxy, desired_state, break_on_error=True) -> None:
    """Poll a tango device until either the given observation state is reached, or it throws an error.
    Arguments:
    device -- Tango Device to check
    desired_state -- The state which to break upon getting (number or state)
    break_on_error -- If set to False, will keeping running when getting an error status.
    """
    spinL = 0
    poll = 1
    while device.obsState != desired_state:
        if spinL < len(spinner) - 1:
            spinL += 1
        else:
            spinL = 0
        sleep(0.5)
        print(
            "\r",
            f"{spinner[spinL]} Poll# {poll}: Current state is {device.obsState.name}, waiting for {desired_state}...",
            end="",
        )
        if device.obsState == 9 and break_on_error:
            break
        poll += 1
    print(f"\nFinished with: {device.obsState.name}")

## 1.2 Set Variables

First, grab the namespace launched from the pipeline:

In [3]:
!kubectl get ns | grep ska-mid-psi

ci-ska-mid-psi-1361134538-alexschell                       Active   103m
ci-ska-mid-psi-1361134538-alexschell-sdp                   Active   104m
ci-ska-mid-psi-1364863847-jaredmda                         Active   34m
ci-ska-mid-psi-1364863847-jaredmda-sdp                     Active   34m


And load it into the variables for the notebook:

In [4]:
namespace = "ci-ska-mid-psi-1361134538-alexschell"  # set to desired NS
simulation_mode = 0  # set to 1 to run in sim mode
target_boards_list = [2]  # assign boards
delay_model_filename = "delay_model.json"  # delay model file to use
test_id = "talon2 basic gaussian noise"  # Test to send config from

Next, we load all the other vars the notebook will use. These should not need to be changed for this run.

In [5]:
# Fully Qualified Domain Names for the devices to set up proxies
CSP_CONTROLLER_FQDN = "mid-csp/control/0"
CSP_SUBARRAY_FQDN = "mid-csp/subarray/01"

CENTRAL_NODE_FQDN = "ska_mid/tm_central/central_node"
LEAF_NODE_FQDN = "ska_mid/tm_leaf_node/csp_master"
LEAF_NODE_SUBARRAY_FQDN = "ska_mid/tm_leaf_node/csp_subarray01"
TMC_SUBARRAY_FQDN = "ska_mid/tm_subarray_node/1"

CBF_CONTROLLER_FQDN = "mid_csp_cbf/sub_elt/controller"
CBF_SUBARRAY_FQDN = "mid_csp_cbf/sub_elt/subarray_01"

BITE_FQDN = "mid_csp_cbf/ec/bite"
DEPLOYER_FQDN = "mid_csp_cbf/ec/deployer"

# Tango host environment variable
TANGO_HOST = "databaseds-tango-base." + namespace + ".svc.cluster.local:10000"

# Parent directory to use to grab config files.
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data")
# Config file directories
COMMON_CONFIG = os.path.join(DATA_DIR, "mid_telescope/cbf")
CSP_CONFIG = os.path.join(DATA_DIR, "mid_telescope/csp")
TMC_CONFIG = os.path.join(DATA_DIR, "mid_telescope/tmc")
HW_CONFIG = os.path.join(COMMON_CONFIG, "hw_config")
SLIM_CONFIG = os.path.join(COMMON_CONFIG, "slim_config")
CBF_INPUT_DIR = os.path.join(COMMON_CONFIG, "cbf_input_data")
# For mapping the talon boards to receptor
RECEPTOR_MAP = ["SKA001", "SKA036", "SKA063", "SKA100"]

Next, set the environment arg for TANGO HOST:

In [7]:
print("Will be using HOST: ", TANGO_HOST)
os.environ["TANGO_HOST"] = TANGO_HOST

Will be using HOST:  databaseds-tango-base.ci-ska-mid-psi-1361134538-alexschell.svc.cluster.local:10000


With all the file paths defined we can load in the JSON files and ensure they exist.

In [8]:
print("Getting files...")

INIT_SYS_PARAM_FILE = os.path.join(COMMON_CONFIG, "sys_params/initial_system_param.json")
ASSIGN_RESOURCES_FILE = os.path.join(TMC_CONFIG, "assign_resources.json")
ASSIGN_CSP_RESOURCES_FILE = os.path.join(CSP_CONFIG, "assign_resources.json")
CONFIGURE_SCAN_FILE = os.path.join(TMC_CONFIG, "configure_scan.json")
SCAN_FILE = os.path.join(TMC_CONFIG, "scan.json")
CSP_DELAY_MODEL_FILE = os.path.join(TMC_CONFIG, delay_model_filename)

CBF_INPUT_FILE = f"{CBF_INPUT_DIR}/cbf_input_data.json"
BITE_CONFIG_FILE = f"{CBF_INPUT_DIR}/bite_config_parameters/bite_configs.json"
FILTERS_FILE = f"{CBF_INPUT_DIR}/bite_config_parameters/filters.json"

DISH_CONFIG_FILE = f"{COMMON_CONFIG}/sys_params/load_dish_config.json"

START_CHANNEL = 0
END_CHANNEL = 14860
START_PORT = 21000

SCAN_COMBOS = [[0.0, 5.0], [0.0, -5.0], [5.0, 0.0], [-5.0, 0.0]]

files = [
    INIT_SYS_PARAM_FILE,
    ASSIGN_RESOURCES_FILE,
    ASSIGN_CSP_RESOURCES_FILE,
    ASSIGN_CSP_RESOURCES_FILE,
    CONFIGURE_SCAN_FILE,
    SCAN_FILE,
    CSP_DELAY_MODEL_FILE,
    CBF_INPUT_FILE,
    BITE_CONFIG_FILE,
    FILTERS_FILE,
    DISH_CONFIG_FILE,
]

for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")

print("Done")

Getting files...
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/sys_params/initial_system_param.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/assign_resources.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/csp/assign_resources.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/csp/assign_resources.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/configure_scan.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/scan.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/delay_model.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/cbf_input_data/cbf_input_data.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/cbf_input_data/bite_config_parameters/bite_c

Next, set up the hw config to match the boards in use:

In [9]:
if any(i > 4 for i in target_boards_list):
    print("Using swap for higher number talons")
    config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_boards_list = list(map(lambda x: x - 4, target_boards_list))

else:
    print("Using standard HW config")
    config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG, config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")

receptor_ids = list(map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list))

Using standard HW config
HW config: ✔️


If the HW config file exists, we can load it into the pod:

In [10]:
!kubectl cp $HW_CONFIG_FILE $namespace/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

Next we can run the deployer to get everything needed for this run

### 1.3 Create Device Proxies

Using the FQDNs we set earlier, and with the pod spun up, we can create device proxies to the devices we'll be using and check the connection to them.

In [11]:
# CSP Devices
csp_controller = DeviceProxy(CSP_CONTROLLER_FQDN)
print(f"CSP Controller: {csp_controller.Status()}")
csp_subarray = DeviceProxy(CSP_SUBARRAY_FQDN)
print(f"CSP Subarray: {csp_subarray.Status()}")

# TMC Devices
tmc_central_node = DeviceProxy(CENTRAL_NODE_FQDN)
print(f"Central Node: {tmc_central_node.Status()}")
leaf_node = DeviceProxy(LEAF_NODE_FQDN)
print(f"Leaf Node: {leaf_node.Status()}")
leaf_node_subarray = DeviceProxy(LEAF_NODE_SUBARRAY_FQDN)
print(f"Leaf Subarray Node: {leaf_node_subarray.Status()}")
tmc_subarray = DeviceProxy(TMC_SUBARRAY_FQDN)
print(f"TMC subarray Node: {tmc_subarray.Status()}")

# Deployer for setup and BITE for data mocking
bite = DeviceProxy(BITE_FQDN)
deployer = DeviceProxy(DEPLOYER_FQDN)

# CBF Device
cbf_controller = DeviceProxy(CBF_CONTROLLER_FQDN)
print(f"CBF controller: {cbf_controller.Status()}")
cbf_subarray = DeviceProxy(CBF_SUBARRAY_FQDN)
print(f"CBF subarray: {cbf_subarray.Status()}")

CSP Controller: The device is in DISABLE state.
CSP Subarray: The device is in DISABLE state.
Central Node: The device is in ON state.
Leaf Node: The device is in ON state.
Leaf Subarray Node: The device is in ON state.
TMC subarray Node: The device is in ON state.
CBF controller: The device is in DISABLE state.
CBF subarray: The device is in DISABLE state.


### 1.4 Downloading Requirements via the Deployer

First, set the board we'll be deploying to and turn on the deployer device.

In [ ]:
deployer.On()
deployer.targetTalons = target_boards_list
print(deployer.targetTalons)
deployer.generate_config_jsons()

Once started and configured, the required devices can then be downloaded.

In [ ]:
deployer.set_timeout_millis(400000)
try:
    deployer.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer.set_timeout_millis(3000)

Now configure the device database with the downloaded devices:

In [ ]:
deployer.configure_db()

## 2 Setting up the Devices

### Start up SDP

In [12]:
# Set up SDP TANGO device proxy
sdp = DeviceProxy("test-sdp/subarray/01")
sdp.On()
print(sdp.Status())

DevFailed: DevFailed[
DevError[
    desc = Command not allowed in device state ON
  origin = command_allowed_state
  reason = API_CommandNotAllowed
severity = ERR]

DevError[
    desc = Failed to execute command_inout on device test-sdp/subarray/01, command On
  origin = virtual Tango::DeviceData Tango::Connection::command_inout(const string&, const Tango::DeviceData&) at (/src/cppTango/cppapi/client/devapi_base.cpp:1338)
  reason = API_CommandFailed
severity = ERR]
]

In [13]:
with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    resources_json = json.load(f)
resources_json["dish"]["receptor_ids"] = list(
    map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list)
)
resources_json["sdp"]["resources"]["receptors"] = list(
    map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list)
)

sdp_only_json = resources_json["sdp"]
print("SDP resources JSON:")
print(json.dumps(sdp_only_json, indent=1))
print("================================")

SDP resources JSON:
{
 "interface": "https://schema.skao.int/ska-sdp-assignres/0.4",
 "resources": {
  "csp_links": [
   1,
   2,
   3,
   4
  ],
  "receptors": [
   "SKA036"
  ],
  "receive_nodes": 1
 },
 "execution_block": {
  "eb_id": "eb-test-20210630-00054",
  "context": {},
  "max_length": 21600.0,
  "channels": [
   {
    "channels_id": "vis_channels",
    "spectral_windows": [
     {
      "spectral_window_id": "fsp_1_channels",
      "count": 14880,
      "start": 0,
      "stride": 1,
      "freq_min": 300000000.0,
      "freq_max": 550000000.0,
      "link_map": [
       [
        0,
        0
       ],
       [
        200,
        1
       ],
       [
        744,
        2
       ],
       [
        944,
        3
       ]
      ]
     }
    ]
   }
  ],
  "polarisations": [
   {
    "polarisations_id": "all",
    "corr_type": [
     "XX",
     "XY",
     "YX",
     "YY"
    ]
   }
  ],
  "fields": [
   {
    "field_id": "field_a",
    "phase_dir": {
     "ra": [
      2.7

In [14]:
print(sdp.ObsState)

obsState.EMPTY


In [ ]:
sdp.ObsReset()

In [15]:
sdp.set_timeout_millis(100000)
sdp.AssignResources(json.dumps(sdp_only_json))
wait_for_state(sdp, 2)

 ⣾ Poll# 136: Current state is FAULT, waiting for 2... 2...
Finished with: FAULT


In [16]:
with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    sdp_configuration = json.load(f)
    sdp_configuration = sdp_configuration["sdp"]
print("SDP config JSON:")
print(json.dumps(sdp_configuration))
print("========================")

SDP config JSON:
{"interface": "https://schema.skao.int/ska-sdp-configure/0.4", "scan_type": "science"}


In [ ]:
# Pass the JSON configuration to the device after ensuring it is correct, waiting to ensure it goes to READY
sdp.Configure(json.dumps(sdp_configuration))
wait_for_state(sdp, 4)

### 2.1 Start up CSP Controller

Now setup the CSP proxy, and load in the relevant values:

In [17]:
# Set relevant values on the mid-csp controller
csp_controller.adminMode = 0
csp_subarray.adminMode = 0
sleep(1)
csp_controller.cbfSimulationMode = simulation_mode
csp_subarray.SimulationMode = simulation_mode
sleep(1)
if (
    csp_controller.read_attribute("adminMode").value == 0
    and csp_controller.read_attribute("cbfSimulationMode").value == 0
):
    print("Set to simulation mode off.")
elif (
    csp_controller.read_attribute("adminMode").value == 0
    and csp_controller.read_attribute("cbfSimulationMode").value == 1
):
    print("Set to simulation mode on.")
else:
    print("Error, couldn't set values!")

Set to simulation mode off.


Next we can load in the dish config file to the central node:

In [18]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = f.read()
tmc_central_node.LoadDishCfg(dish_config_json)

[array([2], dtype=int32), ['1720460575.7055192_159857589088445_LoadDishCfg']]

Now, load in the initial parameters file:

In [19]:
with open(INIT_SYS_PARAM_FILE, encoding="utf-8") as init_file:
    data = json.load(init_file)
print("Initial system parameter file:")
print(json.dumps(data, indent=1))

Initial system parameter file:
{
 "interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0",
 "dish_parameters": {
  "SKA001": {
   "vcc": 1,
   "k": 1108
  },
  "SKA036": {
   "vcc": 2,
   "k": 1108
  },
  "SKA063": {
   "vcc": 3,
   "k": 1100
  },
  "SKA100": {
   "vcc": 4,
   "k": 1101
  }
 }
}


In [20]:
# After confirming the file is correct load into the CBF
cbf_controller.InitSysParam(json.dumps(data))

[array([0], dtype=int32), ['CbfController InitSysParam command completed OK']]

### 2.2 Assign Resources to CSP

In [21]:
with open(ASSIGN_CSP_RESOURCES_FILE, encoding="utf-8") as init_file:
    config_dict = json.load(init_file)
config_dict["dish"]["receptor_ids"] = receptor_ids
print(json.dumps(config_dict, indent=1))

{
 "interface": "https://schema.skao.int/ska-csp-assignresources/2.2",
 "subarray_id": 1,
 "dish": {
  "receptor_ids": [
   "SKA036"
  ]
 }
}


In [22]:
csp_subarray.On()

[array([2], dtype=int32), ['1720460639.8490546_260651285562870_On']]

In [23]:
csp_subarray.AssignResources(json.dumps(config_dict))
wait_for_state(csp_subarray, 2)

 ⣽ Poll# 1: Current state is IDLE, waiting for 2...
Finished with: IDLE


### 2.3 Turning on the Telescope

We can now check if the VCC mapping was loaded

In [28]:
print(leaf_node.sourceDishVccConfig)

{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "tm_data_sources": ["car://gitlab.com/ska-telescope/ska-telmodel-data?ska-sdp-tmlite-repository-1.0.0#tmdata"], "tm_data_filepath": "instrument/ska1_mid_psi/ska-mid-cbf-system-parameters.json"}


Now, turn the telescope itself on:

In [29]:
tmc_central_node.set_timeout_millis(100000)
tmc_central_node.TelescopeOn()
sleep(100)

print(tmc_central_node.status())
print(csp_controller.status())

The device is in ON state.
The device is in ON state.


In [30]:
print("Verify device states:")
print(f"\tTMC central node: {tmc_central_node.State()}. Should be ON")
print(f"\tTMC subarray node: {tmc_subarray.State()}. Should be ON")
print(f"\tCSP controller node: {csp_controller.State()}. Should be ON")
print(f"\tCSP subarray node: {csp_subarray.State()}. Should be ON")
print(f"\tCBF subarray node: {cbf_subarray.State()}. Should be ON")

Verify device states:
	TMC central node: ON. Should be ON
	TMC subarray node: ON. Should be ON
	CSP controller node: ON. Should be ON
	CSP subarray node: ON. Should be ON
	CBF subarray node: ON. Should be ON


## 3 Controlling the 5-Point Scan from TMC

### 3.1 Set up JSON Files

In [31]:
with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as assign_file:
    basic_assign_json = json.load(assign_file)

Load in the config file for TMC

In [32]:
with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as config_file:
    basic_configure_json = json.load(config_file)

Next we grab the IP of the pod to use

In [34]:
# Grab the pod name
!kubectl -n $namespace-sdp get pods | grep vis-receive

In [ ]:
# Then find the IP:
vis_pod = "proc-pb-test-20211111-00059-vis-receive-00-0"
!kubectl -n $ns-sdp describe pod $vis_pod | grep net1 -A 3

In [35]:
output_host = "10.50.1.34"

Now overwrite what we'll need to modify in the files:

In [36]:
today = date.today().strftime("%Y%m%d")
random_id = random.randint(0, 99999)
beams = [
    {"beam_id": "vis0", "function": "visibilities"},
    {"beam_id": "pss1", "search_beam_id": 1, "function": "pulsar search"},
    {"beam_id": "pss2", "search_beam_id": 2, "function": "pulsar search"},
    {"beam_id": "pst1", "timing_beam_id": 1, "function": "pulsar timing"},
    {"beam_id": "pst2", "timing_beam_id": 2, "function": "pulsar timing"},
    {"beam_id": "vlbi1", "vlbi_beam_id": 1, "function": "vlbi"},
]

port_list = []
run = 0
while run * 20 + START_CHANNEL <= END_CHANNEL:
    port_list.append([run * 20 + START_CHANNEL, START_PORT + run])
    run += 1

EB_ID = f"eb-testtmc-{today}-{random_id:05d}"
PB_ID = f"pb-testrealtime-{today}-{random_id:05d}"

# Configure the assign resources JSON for TMC
basic_assign_json["dish"]["receptor_ids"] = receptor_ids
basic_assign_json["sdp"]["execution_block"]["beams"] = beams
basic_assign_json["sdp"]["processing_blocks"][0]["pb_id"] = PB_ID
basic_assign_json["sdp"]["execution_block"]["eb_id"] = EB_ID

# Configure the configuration JSON for TMC
basic_configure_json["csp"]["common"]["config_id"] = "1 receptor, band 1, 1 FSP, no options"
basic_configure_json["csp"]["subarray"]["subarray_name"] = "1 receptor"

# Set the FSP ID to match the board and set networking
basic_configure_json["csp"]["cbf"]["fsp"][0]["fsp_id"] = target_boards_list[0]
basic_configure_json["csp"]["cbf"]["fsp"][0]["output_host"][0][0] = 0
basic_configure_json["csp"]["cbf"]["fsp"][0]["output_host"][0][1] = output_host
basic_configure_json["csp"]["cbf"]["fsp"][0]["output_port"] = port_list


print("Modified assign json:")
print(json.dumps(basic_assign_json, indent=1))
print("===================")
print(json.dumps(basic_configure_json, indent=1))
print("===================")

Modified assign json:
{
 "interface": "https://schema.skao.int/ska-tmc-assignresources/2.1",
 "transaction_id": "txn-....-00001",
 "subarray_id": 1,
 "dish": {
  "receptor_ids": [
   "SKA036"
  ]
 },
 "sdp": {
  "interface": "https://schema.skao.int/ska-sdp-assignres/0.4",
  "resources": {
   "csp_links": [
    1,
    2,
    3,
    4
   ],
   "receptors": [
    "SKA001"
   ],
   "receive_nodes": 1
  },
  "execution_block": {
   "eb_id": "eb-testtmc-20240708-71523",
   "context": {},
   "max_length": 21600.0,
   "channels": [
    {
     "channels_id": "vis_channels",
     "spectral_windows": [
      {
       "spectral_window_id": "fsp_1_channels",
       "count": 14880,
       "start": 0,
       "stride": 1,
       "freq_min": 300000000.0,
       "freq_max": 550000000.0,
       "link_map": [
        [
         0,
         0
        ],
        [
         200,
         1
        ],
        [
         744,
         2
        ],
        [
         944,
         3
        ]
       ]
      }


### 3.2 Assign Resources to the Telescope and Configure Subarray

In [37]:
tmc_subarray.AssignResources(json.dumps(basic_assign_json))
wait_for_state(tmc_subarray, 2)

 ⣟ Poll# 5: Current state is IDLE, waiting for 2...r 2...
Finished with: IDLE


In [39]:
tmc_subarray.Configure(json.dumps(basic_configure_json))
wait_for_state(tmc_subarray, 2)


Finished with: IDLE


### 3.3 Setup BITE Data and Generate

For this version of the notebook, we look to BITE to generate the data we'll use for scanning.

First, load in all the data we'll use

In [40]:
with open(CBF_INPUT_FILE, encoding="utf-8") as f:
    cbf_input_json = json.load(f)["cbf_input_data"][test_id]
    cbf_input_data = json.dumps(cbf_input_json)
    print("CBF Input Data used to generate BITE data:\n")
    print(cbf_input_data)

bite.load_cbf_input_data(cbf_input_data)

CBF Input Data used to generate BITE data:

{"receptors": [{"talon": "2", "dish_id": "SKA036", "sample_rate_k": 618, "bite_config_id": "basic gaussian noise 2", "bite_initial_timestamp_time_offset": 60.0}]}


[array([0], dtype=int32), ['Successfully loaded CBF input data.']]

In [41]:
with open(BITE_CONFIG_FILE, encoding="utf-8") as f:
    bite_config_data = json.dumps(json.load(f))
    print("BITE configs:\n")
    print(bite_config_data)

bite.load_bite_config_data(bite_config_data)

BITE configs:

{"bite_configs": {"basic gaussian noise 1": {"description": "4 seconds of basic gaussian noise with no filtering, using the default mean (0) and standard deviation (0.5).", "sources": [{"description": "Noise input, independent between X and Y pols; unique filter shapes for X and Y pols.", "gaussian": {"pol_x": {"seed": 1234, "filter": "filter_all_pass"}, "pol_y": {"seed": 9876, "filter": "filter_all_pass"}}, "pol_coupling_rho": 0.0, "pol_Y_1_sample_delay": false}], "tone_gens": [], "lstv_seconds": 4}, "basic gaussian noise 2": {"description": "4 seconds of basic gaussian noise with no filtering, using the default mean (0) and standard deviation (0.5).", "sources": [{"description": "Noise input, independent between X and Y pols; unique filter shapes for X and Y pols.", "gaussian": {"pol_x": {"seed": 1234, "filter": "filter_all_pass"}, "pol_y": {"seed": 9876, "filter": "filter_all_pass"}}, "pol_coupling_rho": 0.0, "pol_Y_1_sample_delay": false}], "tone_gens": [], "lstv_sec

[array([0], dtype=int32), ['Successfully loaded BITE config data.']]

In [42]:
with open(FILTERS_FILE, encoding="utf-8") as f:
    filter_data = json.dumps(json.load(f))
    print("Filters:\n")
    print(filter_data)

bite.load_filter_data(filter_data)

Filters:

{"num_coeffs": 1024, "fxp_frac_bits": 16, "filters": {"filter_all_pass": {"description": "All pass filter ^^^^^^^^^^", "band_edges": [0.0, 1.0], "band_gains": [1.0, 1.0], "window": {"kaiser": {"beta": 14}}}, "filter_zero_pass": {"description": "Zero pass filter ___________", "band_edges": [0.0, 1.0], "band_gains": [0.0, 0.0], "window": "hann"}, "filter_ramp_up": {"description": "Ascending ramp shaping filter", "band_edges": [0.0, 1.0], "band_gains": [0.0001, 1.0], "window": "hamming"}, "filter_ramp_down": {"description": "Descending ramp shaping filter", "band_edges": [0.0, 1.0], "band_gains": [1.0, 0.0001], "window": "hamming"}, "filter_band_pass": {"description": "band pass filter _______/^^^^^^^^\\________", "band_edges": [0.0, 0.15, 0.15, 0.18, 0.18, 1.0], "band_gains": [0.0, 0.0, 1.0, 1.0, 0.0, 0.0], "window": {"kaiser": {"beta": 14}}}, "filter_low_pass": {"description": "low pass filter 1.0 ^^^^^^^^^\\________ 0.2", "band_edges": [0.0, 0.5, 0.5, 1.0], "band_gains": [1.0

[array([0], dtype=int32), ['Successfully loaded filter data.']]

With these loaded, we can actually generate the BITE data:

In [43]:
# kubectl logs -n $NS ds-bite-bite-0 -f

bite.set_timeout_millis(240000)
bite.generate_bite_data()
bite.set_timeout_millis(3000)

## 4 Running the 5-Point Scan

Now we can actually run the scans:

### Start SDP Scanning

In [44]:
# Set up SDP TANGO device proxy
sdp = DeviceProxy("test-sdp/subarray/01")
sdp.On()
print(sdp.Status())

DevFailed: DevFailed[
DevError[
    desc = Command not allowed in device state ON
  origin = command_allowed_state
  reason = API_CommandNotAllowed
severity = ERR]

DevError[
    desc = Failed to execute command_inout on device test-sdp/subarray/01, command On
  origin = virtual Tango::DeviceData Tango::Connection::command_inout(const string&, const Tango::DeviceData&) at (/src/cppTango/cppapi/client/devapi_base.cpp:1338)
  reason = API_CommandFailed
severity = ERR]
]

In [45]:
with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    resources_json = json.load(f)
resources_json["dish"]["receptor_ids"] = list(
    map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list)
)
resources_json["sdp"]["resources"]["receptors"] = list(
    map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list)
)

sdp_only_json = resources_json["sdp"]

In [46]:
# After confirming the resource assignment is correct, run the actual argument and wait for device to go to IDLE
sdp.set_timeout_millis(10000)
sdp.AssignResources(json.dumps(sdp_only_json))
wait_for_state(sdp, 2)

DevFailed: DevFailed[
DevError[
    desc = Command not allowed in obsState FAULT
  origin = command_allowed_obs_state
  reason = API_CommandNotAllowed
severity = ERR]

DevError[
    desc = Failed to execute command_inout on device test-sdp/subarray/01, command AssignResources
  origin = virtual Tango::DeviceData Tango::Connection::command_inout(const string&, const Tango::DeviceData&) at (/src/cppTango/cppapi/client/devapi_base.cpp:1338)
  reason = API_CommandFailed
severity = ERR]
]

In [48]:
print(sdp.ObsState)

obsState.FAULT


### Start CSP Replay

Load in the scan file and augment it to match our needs, then use the configure command to pass it in to the CSP.

In [47]:
output_host = "10.50.1.34"

port_list = []
run = 0
while run * 20 + START_CHANNEL <= END_CHANNEL:
    port_list.append([run * 20 + START_CHANNEL, START_PORT + run])
    run += 1

with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    csp_configure_scan = json.load(f)["csp"]
with open(SCAN_FILE, encoding="utf-8") as f:
    csp_scan = json.load(f)
csp_configure_scan["common"]["config_id"] = "1 receptor, band 1, 1 FSP, no options"
csp_configure_scan["subarray"]["subarray_name"] = "1 receptors"
# Write FSP and related data

csp_configure_scan["cbf"]["fsp"][0]["fsp_id"] = target_boards_list[0]
csp_configure_scan["cbf"]["fsp"][0]["zoom_factor"] = 1
csp_configure_scan["cbf"]["fsp"][0]["zoom_window_tuning"] = 450000
csp_configure_scan["cbf"]["fsp"][0]["channel_offset"] = 0
csp_configure_scan["cbf"]["fsp"][0]["receptors"] = list(
    map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list)
)
csp_configure_scan["cbf"]["fsp"][0]["output_host"][0][0] = 0
csp_configure_scan["cbf"]["fsp"][0]["output_host"][0][1] = output_host

csp_configure_scan["cbf"]["fsp"][0]["output_port"] = port_list

print("Modified CSP scan configuration is:")
print(json.dumps(csp_configure_scan, indent=1))
print("========================")
print("CSP scan file is:")
print(json.dumps(csp_scan, indent=1))
print("========================")

Modified CSP scan configuration is:
{
 "interface": "https://schema.skao.int/ska-csp-configure/2.4",
 "subarray": {
  "subarray_name": "1 receptors"
 },
 "common": {
  "config_id": "1 receptor, band 1, 1 FSP, no options",
  "subarray_id": 1,
  "frequency_band": "1"
 },
 "cbf": {
  "fsp": [
   {
    "fsp_id": 2,
    "function_mode": "CORR",
    "frequency_slice_id": 1,
    "zoom_factor": 1,
    "integration_factor": 10,
    "output_link_map": [
     [
      0,
      1
     ]
    ],
    "channel_offset": 0,
    "zoom_window_tuning": 450000,
    "output_host": [
     [
      0,
      "10.50.1.34"
     ]
    ],
    "output_port": [
     [
      0,
      21000
     ],
     [
      20,
      21001
     ],
     [
      40,
      21002
     ],
     [
      60,
      21003
     ],
     [
      80,
      21004
     ],
     [
      100,
      21005
     ],
     [
      120,
      21006
     ],
     [
      140,
      21007
     ],
     [
      160,
      21008
     ],
     [
      180,
      2100

In [50]:
# After confirming the configure scan file is expected, pass to the csp subarray configure command
csp_subarray.Configure(json.dumps(csp_configure_scan))
sleep(5)
print("CBF subarray Observation state after running Configure: {}".format(csp_subarray.obsState))
wait_for_state(csp_subarray, 4)

CBF subarray Observation state after running Configure: 4

Finished with: READY


In [51]:
# Configure the leaf node
leaf_node_subarray.Configure(json.dumps(csp_configure_scan))
sleep(5)
print("CBF subarray Observation state after running Configure: {}".format(csp_subarray.obsState))
wait_for_state(csp_subarray, 4)

CBF subarray Observation state after running Configure: 4

Finished with: READY


Once this is done, we can start the LSTV replay, making sure to record the epoch value provided by it

In [54]:
bite.start_lstv_replay()

[array([0], dtype=int32), ['Successfully started LSTV replay.']]

In [55]:
target_epoch = 773776878.0

With the target epoch ready, we can load it into CSP's delay model.

In [56]:
with open(CSP_DELAY_MODEL_FILE, encoding="utf-8") as f:
    delay_model = json.load(f)
delay_model["start_validity_sec"] = target_epoch
print("Delay model JSON:")
print(json.dumps(delay_model, indent=1))
print("========================")

Delay model JSON:
{
 "interface": "https://schema.skao.int/ska-csp-delaymodel/3.0",
 "start_validity_sec": 773776878.0,
 "cadence_sec": 400.0,
 "validity_period_sec": 400.0,
 "config_id": "sbi-mvp01-20200325-00001-science_A",
 "subarray": 1,
 "receptor_delays": [
  {
   "receptor": "SKA001",
   "xypol_coeffs_ns": [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
   ],
   "ypol_offset_ns": 0.0
  }
 ]
}


In [57]:
delayModelProxy = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray_01")
delayModelProxy.write_attribute("delayModel", json.dumps(delay_model))

### 4.1 Run Scan 1: No Offset

First, we have the subarray scan without setting an offset, such that it scans at [0,0].

In [58]:
long_basic_scan_json = {
    "interface": "https://schema.skao.int/ska-tmc-scan/2.1",
    "transaction_id": "txn-....-00003",
    "scan_id": 1,
}

In [61]:
leaf_node_subarray.Scan(json.dumps(long_basic_scan_json))
wait_for_state(cbf_subarray, 5)
print("Central Scan has started, giving it some time to run...")
sleep(100)
leaf_node_subarray.EndScan()
wait_for_state(cbf_subarray, 4)

 ⣽ Poll# 1: Current state is SCANNING, waiting for 5...
Finished with: SCANNING
Central Scan has started, giving it some time to run...
 ⣽ Poll# 1: Current state is READY, waiting for 4...
Finished with: READY


Once this passes, the offset scans can be run.

### 4.2 Run Offset Scans 2-5

In [62]:
leaf_node_subarray.Configure(json.dumps(csp_configure_scan))
wait_for_state(cbf_subarray, 4)


Finished with: READY


In [63]:
print(cbf_subarray.ObsState)

obsState.READY


Now, loop through the offset scans, using the combos to set the scan json. For each, we configure by passing in JSON, then scan again, letting it run before stopping.

In [64]:
count = 4
for offset in SCAN_COMBOS:
    print(f"setting offset to: {offset}")

    # Configure for this offset scan
    partial_configure_json = {
        "interface": "https://schema.skao.int/ska-tmc-configure/2.2",
        "transaction_id": f"txn-....-0000{count}",
        "scan_id": count,
        "pointing": {"target": {"ca_offset_arcsec": offset[0], "ie_offset_arcsec": offset[1]}},
        "tmc": {"partial_configuration": True},
    }
    # print(json.dumps(partial_configure_json, indent=1))
    print(".......")
    leaf_node_subarray.Configure(json.dumps(csp_configure_scan))
    csp_subarray.Configure(json.dumps(csp_configure_scan))
    wait_for_state(cbf_subarray, 4)

    # Send the Scan command along with relevant JSON
    partial_scan_json = {
        "interface": "https://schema.skao.int/ska-tmc-scan/2.1",
        "transaction_id": f"txn-....-0000{count+4}",
        "scan_id": 2 + count,
    }
    leaf_node_subarray.Scan(json.dumps(partial_scan_json))
    wait_for_state(cbf_subarray, 5)
    sleep(200)
    leaf_node_subarray.EndScan()
    wait_for_state(cbf_subarray, 4)

    count += 1
    print("============================")
print("Done offsets!")

setting offset to: [0.0, 5.0]
.......

Finished with: READY


DevFailed: DevFailed[
DevError[
    desc = ska_tmc_common.exceptions.InvalidObsStateError: Scan command is not allowed in current observation state on device mid-csp/subarray/01
           Reason: The current observation state of mid-csp/subarray/01 for observation is 3
           The Scan command has NOT been executed. This device will continue with normal operation.
           
  origin = Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tango/device_server.py", line 85, in wrapper
    return get_worker().execute(fn, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tango/green.py", line 101, in execute
    return fn(*args, **kwargs)
  File "/app/src/ska_tmc_cspsubarrayleafnode/csp_subarray_leaf_node.py", line 323, in is_Scan_allowed
    return self.component_manager.is_command_allowed("Scan")
  File "/app/src/ska_tmc_cspsubarrayleafnode/manager/component_manager.py", line 563, in is_command_allowed
    self.raise_invalid_obsstate_error

In [ ]:
leaf_node_subarray.EndScan()

## 5 Cleanup:

In [ ]:
tmc_subarray.GoToIdle()

In [ ]:
tmc_subarray.ReleaseAllResources()

In [ ]:
tmc_subarray.ObsReset()